In [ ]:
# Part 4
part_num = 4

# 构建文件名
input_file = f'/content/drive/MyDrive/dsaa5002_project/Merged_News_Part_{part_num}.xlsx'
output_file = f'/content/drive/MyDrive/dsaa5002_project/Result_{part_num}.xlsx'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from transformers import AutoModelForSequenceClassification, BertTokenizer
import torch
from tqdm import tqdm

# 假设df是您的数据框
df = pd.read_excel(input_file)

In [ ]:
# 加载模型和分词器
tokenizer = BertTokenizer.from_pretrained('IDEA-CCNL/Erlangshen-Roberta-110M-Sentiment')
model = AutoModelForSequenceClassification.from_pretrained('IDEA-CCNL/Erlangshen-Roberta-110M-Sentiment')

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/785 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/409M [00:00<?, ?B/s]

In [5]:
# 定义批处理大小
batch_size = 128  # 根据您的计算资源调整

# 函数来处理一个批次的数据
def process_batch(texts):
    inputs = tokenizer(texts, padding=True, truncation=True, max_length=512, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    return torch.nn.functional.softmax(outputs.logits, dim=-1)

# 使用tqdm来创建一个进度条
results = []
for i in tqdm(range(0, len(df), batch_size), desc="Processing batches"):
    batch_texts = df['NewsContent'][i:i+batch_size].tolist()
    batch_results = process_batch(batch_texts)
    results.extend(batch_results)

# 将结果添加到数据框
df['label'] = [result.argmax().item() for result in results]

Processing batches: 100%|██████████| 380/380 [7:59:22<00:00, 75.69s/it]


In [6]:
df.to_excel(output_file)